## Equal and alike quantum expressions using Qbit varaibles

As per QUBO optimization function f(x,y) = x + y - 2xy for equal operator (a unary operation), dann5.d5o2 supports equal expressions for Qbit variables by implementing == operator for Qbit. In the following code we see:
- xEq is a python variable with a reference to QbitExpr (expression) object
- xEq contians x equal y expression, were both x and y are Qbit variables in S(uperposition) state
- xEq expression can be transformed into a QUBO presentation as per the expression's optimization function, whcih can be presented as an equal expression QUBO triangle matrix
    >                   x   y
    >
    >               x   1  -2
    >
    >               y   0   1
- xEq can be solved by finding minimum of optimization function represented by the expression QUBO, which for x equal y has 2 possible solutions:
    > x == y == 0
    >
    > x == y == 1

In [1]:
from dann5.d5o2 import Qbit
x = Qbit("x"); y = Qbit("y")
xEq = x == y
print(xEq)
print("\nEqual Expression:\n", xEq.toString())
print("\nEqual Expression QUBO:\n", xEq.qubo())
print("\nEqual Expression SOLUTION:\n", xEq.solve())


Equal Expression:
 (x/S/ == y/S/)

Equal Expression QUBO:
 {('x', 'x'): 1.0, ('x', 'y'): -2.0, ('y', 'y'): 1.0}

Equal Expression SOLUTION:
 y/0/; x/0/
y/1/; x/1/



In addition to an equal operator it is possible to use alike (not-xor) operation to compare wether values of two quantum Qbit variables are equal, as in the following example.
- xAl references x alike y operation expression, which is syntactically and semantically same as x not-xor y operation expression referenced by xNxor python variable
- both of these QbitExpr objects, xAl and xNxor, can be expressed using same QUBO transformation:
    >                   x   y  *=   #
    >
    >               x  -1   2   2  -4
    >
    >               y   0  -1   2  -4
    >
    >              *=   0   0  -1  -4
    >
    >               #   0   0   0   8
    Where x and y are input variables of an operation, \*= represents a result variable of the not-xor (alike) operation, and # represents a carryforward variable of the operation.

- both of these expressions have the same solution:
    > _*=0, a result temporary variable of the alike expression, xAl, is 1 when x and y have the same value, 0 or 1, and it is 0 when x and y are different, e.g. x is 1 and y is 0, or x is 0 and y is 1.
    >
    > Likewise, _*=1 result temporary variable of not-xor expression, xNxor, is 1 when x and y are same, and it is 0 when they are different.

In [2]:
xAl = x.alike(y)
xNxor = x.nxor(y)
print("Alike Expression QUBO:\n", xAl.qubo())
print("\nNot-xor Expression QUBO:\n", xNxor.qubo())
print("\nAlike Expression SOLUTION:\n", xAl.solve())
print("\nNot-xor Expression SOLUTION:\n", xNxor.solve())

Alike Expression QUBO:
 {('#0', '#0'): 8.0, ('_*=0', '#0'): -4.0, ('_*=0', '_*=0'): -1.0, ('x', '#0'): -4.0, ('x', '_*=0'): 2.0, ('x', 'x'): -1.0, ('x', 'y'): 2.0, ('y', '#0'): -4.0, ('y', '_*=0'): 2.0, ('y', 'y'): -1.0}

Not-xor Expression QUBO:
 {('#1', '#1'): 8.0, ('_*=1', '#1'): -4.0, ('_*=1', '_*=1'): -1.0, ('x', '#1'): -4.0, ('x', '_*=1'): 2.0, ('x', 'x'): -1.0, ('x', 'y'): 2.0, ('y', '#1'): -4.0, ('y', '_*=1'): 2.0, ('y', 'y'): -1.0}

Alike Expression SOLUTION:
 _*=0/0/; x/0/; y/1/
_*=0/0/; x/1/; y/0/
_*=0/1/; x/0/; y/0/
_*=0/1/; x/1/; y/1/


Not-xor Expression SOLUTION:
 _*=1/0/; x/0/; y/1/
_*=1/0/; x/1/; y/0/
_*=1/1/; x/0/; y/0/
_*=1/1/; x/1/; y/1/



## Quantum assignments of Qbit variables
Even though an equal quantum expression and a quantum assignment might result in the same solution, for development of a quantum program it important to recognise the difference between these two. 
> An equal quantum operator allows definition of a comparison expression between two quantum variables. 

> An assignment allows programmer to define a relationship between an expected result variable and an expression that describes a problem.
>    > r = x, is an assignment where r variable represents a solution of a simple expression that has only one variable x. 

- In a way, an assignment of x to r variables means that r and x are the same, and they are interpreted as same enforcing a result variable r, and replacing (ignoring) same variable x. So, we can apply following transformations:
    > r = x <=> r = x == r
    >     => r = r == r
- As a result the QUBO transformation of an asignment is quite simle transformation
    >                   r
    >
    >               r   0
- The solution is according to our expectations:
    > r = x == r == 0
    >
    > r = x == r == 1

In [3]:
r = Qbit("r")
aR = r.assign(x)
print("Assignment:\n", aR.toString())
print("\nAssignment QUBO:\n", aR.qubo())
print("\nAssignment SOLUTION:\n", aR.solve())

Assignment:
 r/S/ = (r/S/ == r/S/)

Assignment QUBO:
 {('r', 'r'): 0.0}

Assignment SOLUTION:
 r/0/; r/0/
r/1/; r/1/



## Equal and alike Qbit expressions in quantum assignment
We can use equal operator to define a quantum expression that is assigned to a result Qbit variable.
- The quantum assignment r = x == y, results in y and r Qbit variables being recognised as same. 
- This is correct as an equal operator is a unary operation and QUBO transformation of a unary operation expression, treats the second operand (in this case y Qbit variable) as an output of the expression.
- So the solution is:
    > r = y = x == r == 0
    >
    > r = y = x == r == 1

In [4]:
axEq = r.assign(x == y)
print("Equal Assignment:\n", axEq.toString())
print("\nEqual Assignment QUBO:\n", axEq.qubo())
print("\nEqual Assignment SOLUTION:\n", axEq.solve())

Equal Assignment:
 r/S/ = (x/S/ == r/S/)

Equal Assignment QUBO:
 {('r', 'r'): 1.0, ('x', 'r'): -2.0, ('x', 'x'): 1.0}

Equal Assignment SOLUTION:
 r/0/; x/0/
r/1/; x/1/



Alike (not-xor) is a binary operation, which has two input operators, in example below Qbit variables x and y, and a result output operand, represented with _*=(plus-number) and its carryforward operand, represented with #(plus-number). When alike (not-xor) expression is assigned to a result Qbit variable, the operation result output binds to (is replaced with) the assignment result variable.
- axAl references x alike (not-xor) y operation expression, which is assigned to a result Qbit variable r. The Qbit alike (not-xor) operation is represented with *= simbol. 
- As the the alike expression temporary result variable is replaced with defined result variable r, the QUBO transformation can we represented with teh same triangular QUBO matrix above, where *= output is replacd with r Qbit result variable .
- The assignment, as the two expressions earlier, have the same solution:
    > r, a Qbit result of the alike assignment, axAl, is 1 when x and y have the same value, 0 or 1, and 
    >
    > it is 0 when x and y are different, e.g. x is 1 and y is 0, or x is 0 and y is 1.

In [5]:
axAl = r.assign(x.alike(y))
print("Alike Assignment:\n", axAl.toString())
print("\nAlike Assignment QUBO:\n", axAl.qubo())
print("\nAlike Assignment SOLUTION:\n", axAl.solve())

Alike Assignment:
 r/S/ = (x/S/ *= y/S/)

Alike Assignment QUBO:
 {('#2', '#2'): 8.0, ('r', '#2'): -4.0, ('r', 'r'): -1.0, ('x', '#2'): -4.0, ('x', 'r'): 2.0, ('x', 'x'): -1.0, ('x', 'y'): 2.0, ('y', '#2'): -4.0, ('y', 'r'): 2.0, ('y', 'y'): -1.0}

Alike Assignment SOLUTION:
 r/0/; x/0/; y/1/
r/0/; x/1/; y/0/
r/1/; x/0/; y/0/
r/1/; x/1/; y/1/



## Not-qual vs. unlike quantum expressions and assignments
Not-equal is a Qbit operator (unary operation), which ensures two Qbit variables, x and y, are different.

In [6]:
xNe = x != y
print("Not-equal Expression:\n", xNe.toString())
print("\nNot-equal Expression QUBO:\n", xNe.qubo())
print("\nNot-equal Expression SOLUTION:\n", xNe.solve())

Not-equal Expression:
 (x/S/ != y/S/)

Not-equal Expression QUBO:
 {('x', 'x'): -1.0, ('x', 'y'): 2.0, ('y', 'y'): -1.0}

Not-equal Expression SOLUTION:
 y/1/; x/0/
y/0/; x/1/



An expression x ^ y (x xor y) is same as x.unlike(y). Similarly to alike (not-xor) Qbit operation, the unlike (xor) binary operation has two input operands x and y, and two output operands, teporary result varaible _^ and carryforward variabel #, as represented by its QUBO. The solution of unlike (xor) operation expression is oposite to that of alike expression:
    > _^, a result temporary variable of the alike expression, xAl, is 0 when x and y have the same value, 0 or 1, and 
    >
    > it is 1 when x and y are different, e.g. x is 1 and y is 0, or x is 0 and y is 1.

In [7]:
xUl = x ^ y
print("Unlike Expression: {}\n\tDecomposed:{}\n".format(xUl.toString(),xUl.toString(True)))
print("\nUnlike Expression QUBO:\n", xUl.qubo())
print("\nUnlike Expression SOLUTION:\n", xUl.solve())

Unlike Expression: (x/S/ ^ y/S/)
	Decomposed:_^0/S/ = x/S/ ^ y/S/; 


Unlike Expression QUBO:
 {('#3', '#3'): 4.0, ('_^0', '#3'): 4.0, ('_^0', '_^0'): 1.0, ('x', '#3'): -4.0, ('x', '_^0'): -2.0, ('x', 'x'): 1.0, ('x', 'y'): 2.0, ('y', '#3'): -4.0, ('y', '_^0'): -2.0, ('y', 'y'): 1.0}

Unlike Expression SOLUTION:
 _^0/0/; x/0/; y/0/
_^0/1/; x/0/; y/1/
_^0/1/; x/1/; y/0/
_^0/0/; x/1/; y/1/



We can enforce x and y to be different using unlike (xor) assignment, and by assigning the unlike (xor) expression to a deterministic constant variable with the value 1. By comparing QUBOs in examples above and below, we see that the QUBO transformaion below has been changed by resolving quadratic QUBO elements containing deterministic Qbit variable _ 1 _. The solution for x and y is same as in case of the not-equal expression above.

In [8]:
_1_ = Qbit("_1_", 1)
axUl = _1_.assign(x.unlike(y))
print("Unlike Assignment: ", axUl.toString())
print("\tQUBO:", axUl.qubo())
print("SOLUTION:\n", axUl.solve())

Unlike Assignment:  _1_/1/ = (x/S/ ^ y/S/)
	QUBO: {('#4', '#4'): 8.0, ('x', '#4'): -4.0, ('x', 'x'): -1.0, ('x', 'y'): 2.0, ('y', '#4'): -4.0, ('y', 'y'): -1.0}
SOLUTION:
 _1_/1/; x/0/; y/1/
_1_/1/; x/1/; y/0/



## Use of operators vs. operations in complex Qbit expressions


In [9]:
b = Qbit("b", 1); z = Qbit("z"); _0_ = Qbit("_0_", 0)
qbitAssign = _1_.assign((z != (b & x)) | (z == (y ^ _0_)))
print("\nLOGIC:{}\n\tDecomposed: {}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** Generic Qubo ***\n{}\n".format(qbitAssign.qubo(False))); 
print("*** Finalized Qubo ***\n{}\n".format(qbitAssign.qubo()))


LOGIC:_1_/1/ = ((z/S/ != (b/1/ & x/S/)) | (z/S/ == (y/S/ ^ _0_/0/)))
	Decomposed: _1_/1/ = _&0/S/ | _^2/S/; z/S/ != _&0/S/; _&0/S/ = b/1/ & x/S/; z/S/ == _^2/S/; _^2/S/ = y/S/ ^ _0_/0/; 
*** Generic Qubo ***
{('#5', '#5'): 4.0, ('_&0', '_&0'): 3.0, ('_&0', '_1_'): -2.0, ('_&0', '_^2'): 1.0, ('_0_', '#5'): -4.0, ('_0_', '_0_'): 1.0, ('_0_', '_^2'): -2.0, ('_1_', '_1_'): 1.0, ('_^2', '#5'): 4.0, ('_^2', '_1_'): -2.0, ('_^2', '_^2'): 3.0, ('b', '_&0'): -2.0, ('b', 'b'): 0.0, ('b', 'x'): 1.0, ('x', '_&0'): -2.0, ('x', 'x'): 0.0, ('y', '#5'): -4.0, ('y', '_0_'): 2.0, ('y', '_^2'): -2.0, ('y', 'y'): 1.0, ('z', '_&0'): 2.0, ('z', '_^2'): -2.0, ('z', 'z'): 0.0}

*** Finalized Qubo ***
{('#5', '#5'): 4.0, ('_&0', '_&0'): -1.0, ('_&0', '_^2'): 1.0, ('_^2', '#5'): 4.0, ('_^2', '_^2'): 1.0, ('x', '_&0'): -2.0, ('x', 'x'): 1.0, ('y', '#5'): -4.0, ('y', '_^2'): -2.0, ('y', 'y'): 1.0, ('z', '_&0'): 2.0, ('z', '_^2'): -2.0, ('z', 'z'): 0.0}



The logical statemenet **_1_ = ((z != (b & x)) | (z == (y ^ _0_)))**, where x, y and z are in superposition state, _1_ and b set to value 1 and _0_ set to value 0 is decoomposed into following logical circuites:
>	                                     _1_/1/
>
>	                   _&3/S/	           |	          	_^6/S/
>
>	          z/S/	 !=	   _&3/S/		       z/S/	   == 		_^6/S/
>
>       		        	b/1/	&	x/S/			              y/S/	^	_0_/0/

The _&3 and _^6 are autoamtically generated result variables for and and xor operations respectfully. Both of the auxiliary result variables is set to superposition state, as at least one of the operations' input argument variables is in superposition state.

In [10]:
from dann5.d5o2 import Qanalyzer
analyze = Qanalyzer(qbitAssign.qubo())
print("# of nodes: {}, # of branches: {}\n".format(analyze.nodesNo(),analyze.branchesNo()))

print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))

# of nodes: 6, # of branches: 7

*** SOLUTION ***
_1_/1/; _&0/0/; b/1/; x/0/; z/1/; _^2/1/; y/1/; _0_/0/; z/1/
_1_/1/; _&0/1/; b/1/; x/1/; z/0/; _^2/0/; y/0/; _0_/0/; z/0/



There are only 2 valid solutions enforced by equal and not-equal operators:
1. Solution x/0/, y/1/ and z/1/ is valid:
>	                                     _1_/1/
>
>	                    _&3/0/	           |	          	_^6/1/
>
>	          z/1/	 !=	   _&3/0/		       z/1/	   == 		_^6/1/
>
>       		        	b/1/	&	x/0/			              y/1/	^	_0_/0/
2. Solution x/1/, y/0/ and z/0/ is valid
>	                                     _1_/1/
>
>	                    _&3/1/	           |	          	_^6/0/
>
>	          z/0/	 !=	   _&3/1/		       z/0/	   == 		_^6/0/
>
>       		        	b/1/	&	x/1/			              y/0/	^	_0_/0/

In [11]:
qbitAssign = _1_.assign(((b & x) != z) | (z == (y ^ _0_)))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = (((b/1/ & x/S/) != z/S/) | (z/S/ == (y/S/ ^ _0_/0/)))
_1_/1/ = z/S/ | _^3/S/; _&1/S/ != z/S/; _&1/S/ = b/1/ & x/S/; z/S/ == _^3/S/; _^3/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; z/1/; _&1/0/; b/1/; x/0/; _^3/1/; y/1/; _0_/0/; z/1/



In [12]:
qbitAssign = _1_.assign((z != (b & x)) | ((y ^ _0_) == z))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = ((z/S/ != (b/1/ & x/S/)) | ((y/S/ ^ _0_/0/) == z/S/))
_1_/1/ = _&2/S/ | z/S/; z/S/ != _&2/S/; _&2/S/ = b/1/ & x/S/; _^4/S/ == z/S/; _^4/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; _&2/1/; b/1/; x/1/; z/1/; z/1/; _^4/1/; y/1/; _0_/0/



In [13]:
qbitAssign = _1_.assign(((b & x) != z) | ((y ^ _0_) == z))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = (((b/1/ & x/S/) != z/S/) | ((y/S/ ^ _0_/0/) == z/S/))
_1_/1/ = z/S/ | z/S/; _&3/S/ != z/S/; _&3/S/ = b/1/ & x/S/; _^5/S/ == z/S/; _^5/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; z/1/; _&3/0/; b/1/; x/0/; z/1/; _^5/1/; y/1/; _0_/0/



## Use of alike and unlike Qbit operations instead of eqaul and not-equal operators

In [14]:
qbitAssign = _1_.assign(((b & x).unlike(z) | ((y ^ _0_).alike(z))))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = (((b/1/ & x/S/) ^ z/S/) | (y/S/ ^ _0_/0/))
_1_/1/ = _^6/S/ | _^7/S/; _^6/S/ = _&4/S/ ^ z/S/; _&4/S/ = b/1/ & x/S/; _^7/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; _^6/0/; _&4/0/; b/1/; x/0/; z/0/; _^7/1/; y/1/; _0_/0/
_1_/1/; _^6/1/; _&4/0/; b/1/; x/0/; z/1/; _^7/0/; y/0/; _0_/0/
_1_/1/; _^6/1/; _&4/0/; b/1/; x/0/; z/1/; _^7/1/; y/1/; _0_/0/
_1_/1/; _^6/1/; _&4/1/; b/1/; x/1/; z/0/; _^7/0/; y/0/; _0_/0/
_1_/1/; _^6/1/; _&4/1/; b/1/; x/1/; z/0/; _^7/1/; y/1/; _0_/0/
_1_/1/; _^6/0/; _&4/1/; b/1/; x/1/; z/1/; _^7/1/; y/1/; _0_/0/



1. Solution x/0/, y/0/ and z/1/ is a valid solution
>	                                 _1_/1/
>
>	              _^4/1/	           |	            _*=0/0/
>
>	          _&3/0/	 ^	z/1/		      _^3/0/	*=	z/1/
>
>       b/1/	&	x/0/			    y/0/	^	_0_/0/
2. Solution x/0/, y/0/ and z/0/ is a valid solution
>	                                 _1_/1/
>
>	              _^4/0/	           |	            _*=0/1/
>
>	          _&3/0/	 ^	z/0/		      _^3/0/	*=	z/0/
>
>       b/1/	&	x/0/			    y/0/	^	_0_/0/
3. Solution x/1/, y/1/ and z/0/ is a valid solution
>	                                 _1_/1/
>
>	              _^4/1/	           |	            _*=0/0/
>
>	          _&3/1/	 ^	z/0/		      _^3/1/	*=	z/0/
>
>       b/1/	&	x/1/			    y/1/	^	_0_/0/
4. Solution x/1/, y/0/ and z/0/ is same valid solution as when we used equal and not-equal operators
5. Solution x/0/, y/1/ and z/1/ is same valid solution as when we used equal and not-equal operators
6. Solution x/1/, y/1/ and z/1/ is a valid solution
>	                                 _1_/1/
>
>	              _^4/0/	           |	            _*=0/1/
>
>	          _&3/1/	 ^	z/1/		      _^3/1/	*=	z/1/
>
>       b/1/	&	x/1/			    y/1/	^	_0_/0/

In [15]:
qbitAssign = _1_.assign((z.unlike(b & x) | ((y ^ _0_).alike(z))))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = ((z/S/ ^ (b/1/ & x/S/)) | (y/S/ ^ _0_/0/))
_1_/1/ = _^8/S/ | _^9/S/; _^8/S/ = z/S/ ^ _&5/S/; _&5/S/ = b/1/ & x/S/; _^9/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; _^8/0/; z/0/; _&5/0/; b/1/; x/0/; _^9/1/; y/1/; _0_/0/
_1_/1/; _^8/1/; z/1/; _&5/0/; b/1/; x/0/; _^9/0/; y/0/; _0_/0/
_1_/1/; _^8/1/; z/1/; _&5/0/; b/1/; x/0/; _^9/1/; y/1/; _0_/0/
_1_/1/; _^8/1/; z/0/; _&5/1/; b/1/; x/1/; _^9/0/; y/0/; _0_/0/
_1_/1/; _^8/1/; z/0/; _&5/1/; b/1/; x/1/; _^9/1/; y/1/; _0_/0/
_1_/1/; _^8/0/; z/1/; _&5/1/; b/1/; x/1/; _^9/1/; y/1/; _0_/0/



In [16]:
qbitAssign = _1_.assign(((b & x).unlike(z) | (z.alike(y ^ _0_))))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = (((b/1/ & x/S/) ^ z/S/) | (z/S/ *= (y/S/ ^ _0_/0/)))
_1_/1/ = _^10/S/ | _*=5/S/; _^10/S/ = _&6/S/ ^ z/S/; _&6/S/ = b/1/ & x/S/; _*=5/S/ = z/S/ *= _^11/S/; _^11/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; _^10/1/; _&6/0/; b/1/; x/0/; z/1/; _*=5/0/; z/1/; _^11/0/; y/0/; _0_/0/
_1_/1/; _^10/0/; _&6/0/; b/1/; x/0/; z/0/; _*=5/1/; z/0/; _^11/0/; y/0/; _0_/0/
_1_/1/; _^10/1/; _&6/1/; b/1/; x/1/; z/0/; _*=5/0/; z/0/; _^11/1/; y/1/; _0_/0/
_1_/1/; _^10/1/; _&6/1/; b/1/; x/1/; z/0/; _*=5/1/; z/0/; _^11/0/; y/0/; _0_/0/
_1_/1/; _^10/1/; _&6/0/; b/1/; x/0/; z/1/; _*=5/1/; z/1/; _^11/1/; y/1/; _0_/0/
_1_/1/; _^10/0/; _&6/1/; b/1/; x/1/; z/1/; _*=5/1/; z/1/; _^11/1/; y/1/; _0_/0/



In [17]:
qbitAssign = _1_.assign((z.unlike(b & x) | (z.alike(y ^ _0_))))
print("\nLOGIC:\n{}\n{}".format(qbitAssign.toString(),qbitAssign.toString(True)))
print("*** SOLUTION ***\n{}".format(qbitAssign.solve()))


LOGIC:
_1_/1/ = ((z/S/ ^ (b/1/ & x/S/)) | (z/S/ *= (y/S/ ^ _0_/0/)))
_1_/1/ = _^12/S/ | _*=6/S/; _^12/S/ = z/S/ ^ _&7/S/; _&7/S/ = b/1/ & x/S/; _*=6/S/ = z/S/ *= _^13/S/; _^13/S/ = y/S/ ^ _0_/0/; 
*** SOLUTION ***
_1_/1/; _^12/1/; z/1/; _&7/0/; b/1/; x/0/; _*=6/0/; z/1/; _^13/0/; y/0/; _0_/0/
_1_/1/; _^12/0/; z/0/; _&7/0/; b/1/; x/0/; _*=6/1/; z/0/; _^13/0/; y/0/; _0_/0/
_1_/1/; _^12/1/; z/0/; _&7/1/; b/1/; x/1/; _*=6/0/; z/0/; _^13/1/; y/1/; _0_/0/
_1_/1/; _^12/1/; z/0/; _&7/1/; b/1/; x/1/; _*=6/1/; z/0/; _^13/0/; y/0/; _0_/0/
_1_/1/; _^12/1/; z/1/; _&7/0/; b/1/; x/0/; _*=6/1/; z/1/; _^13/1/; y/1/; _0_/0/
_1_/1/; _^12/0/; z/1/; _&7/1/; b/1/; x/1/; _*=6/1/; z/1/; _^13/1/; y/1/; _0_/0/

